# Log for project in Computer Science

By Anna and Eric

## Day 1, 4/4 - 2019

Today the goal is to understand the project and be prepared for mondays meeting.

Step 

     1) Download UniProt and check out the dataset

     2) Create the dictionary
     
     3) Get tool from Marcus
     
     4) Train a tagger with machine learning
     
So we would like to finish step 1 and start reseach on 2.

---
Today we sat from 13-16. We started by creating a private git repo with this file in it. Then we downloaded the uniprot dataset in xml form. We proceded to try and parse it with BeautifulSoup. The file was too big for that method.

After that we tried with ElementTree instead which can go through each element one by one. This worked and we eventually got a list of proteins (over 200 000).

We got stuck at which dictionary we want to create. Do we want one that's protein -> gene1 gene2 gene3 and thus also mine genes from the xml file. Or is the dictionary something else? For next time this needs to be cleared up in order to move on.

In [ ]:
#Goes through the file uniprot_sprot.xml and saves the names of the proteins
#in protein_names.txt
from xml.etree import ElementTree as ET
filename='uniprot_sprot.xml'
parser = ET.iterparse(filename)
length = 28 #length of header in xml tags
proteins = set()
for event, element in parser:
    tag = element.tag[length:]
    if 'shortName' in tag: # or 'shortName' in tag #fullName
        proteins.add(element.text)
    element.clear()
f = open('protein_names_short.txt', 'w')
f.write('\n'.join(proteins))
f.close()


## Day 2, 8/4 - 2019

Today we had our second meeting with Pierre and Sonja. We got clarification about the biology bit of the project but also some tips going forward. Now we have a Slack and are to get an invite to the Github repo. Also, an invite to Endnote.

TODO: Invite Sonja, Pierre to Overleaf document

Tips:

    1) Search PubMed for 'biomedical text-mining' and 'bio NLP' (Jensen)
    
    2) Get annotation from Hannes

So this week we wish to get the infrastructure up and running.

---
We started by looking at Marcus email and getting his program to work. After a lot of problems with installation of the various packages and programs we finally got it to work, almost 2h later.

Question: What is the file 'disease_index.fst' and how is it created? Should we create this?

---
We now have the basic infrastructure up and running. We have done some tests on running it with our dataset of proteins from UniProt. Also we started looking at BioCreative's datasets, but we'll leave that for next time. 


### Instructions to start JVM (need Marcus repo):

    1) cd [katalog]/mention-index-py4j

    2) mvn package

    3) cd target

    4) java -jar mentions-index-py4j-1.0-SNAPSHOT.jar

In [ ]:
# Install a conda package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install docria
#!{sys.executable} -m pip install py4j
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.printout import options
from docria.algorithm import group_by_span, dominant_right_span, dominant_right
set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    # Here we do the py4j equivalent for new java.io.File(path)
    return jvm.java.io.File(os.path.abspath(path))

In [ ]:
app.buildIndex(get_java_file("protein_name/protein_names_long.txt"), get_java_file("protein_name/protein_names_long.fst"))
indx = app.loadIndex(get_java_file("protein_name/protein_names_long.fst"))

doc = Document()
doc.add_text("main","Peroxydase reaction stains were negative, chloroacetate esterase were strongly positive.")
binary_doc = MsgpackCodec.encode(doc) 
search_binary_doc = app.search(indx, binary_doc)
doc = MsgpackCodec.decode(search_binary_doc)

tuls = []
for term in doc["matches"]:
    tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
    #print(str(term["text"]), "-", "%d:%d" % (term["text"].start, term["text"].stop))
print(tuls)
dominant_right(tuls)


## Day 3, 9/4-2019
We have had some questions about the purpose and think we have finally figured it out. To identify the entities 'protein' and 'diseases' like cell death. To start with we will identify proteins by checking a pubmed article and using our uniprot list.



In [ ]:
#Goes through the file pubmed19n0651.xml and saves the abstract text as one textfile
from xml.etree import ElementTree as ET
filename='pubmed19n0651.xml'
out_name = 'abstract_text19n0651.txt'
parser = ET.iterparse(filename)
f=open(out_name, "a+", encoding="utf-8")
for event, element in parser:
    if element.tag == 'AbstractText':
        if element.text:
            f.write(element.text)
    element.clear()
f.close()

In [ ]:
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.printout import options
set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    return jvm.java.io.File(os.path.abspath(path))

In [ ]:
#Prints all occurences of proteins in the abstract file.
app.buildIndex(get_java_file("protein_names_long.txt"), get_java_file("protein_names_long.fst"))
indx = app.loadIndex(get_java_file("protein_names_long.fst"))
data = ''
with open('abstract_text19n0651.txt', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('.')
for s in sp:
    hits = indx.search(s)
    for hit in hits:
        print(hit.start, hit.end, " ".join([term.raw for term in hit.terms]))

In [ ]:
#Prints all occurences of cell deaths in the abstract file.
app.buildIndex(get_java_file("cell_death_names.txt"), get_java_file("cell_name_index.fst"))
indx = app.loadIndex(get_java_file("cell_name_index.fst"))
data = ''
with open('abstract_text19n0651.txt', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('.')
for s in sp:
    hits = indx.search(s)
    for hit in hits:
        print(hit.start, hit.end, " ".join([term.raw for term in hit.terms]))

Next we tried to use the model that https://aclweb.org/anthology/W16-5104 implemented at github: https://github.com/withtwist/medical-ner. After transpiling the code to Python 2, changing the encoding of the data etc. we still got compilation errors. It seemed too big of a job so we scraped it. Since we have been able to identify proteins from the abstract and where they are located, we are now interested in finding a way to create the tracer with machine learning. We found the article *Unsupervised biomedical named entity recognition: Experiments with clinical and biological texts. 


Note: We noticed that the indx.search is case sensitive, which needs to be fixed. (e.g. toLowerCase or similar). Check w Marcus!!

Conclusion of todays work: We have fixed the dictionaries and spent some time making sure the xml reader for uniprot included the edgecases and included 'shortName'. We also started tinkering with neural networks by downloading existing repos but we could not make it work. We started reading a couple of papers to get inspiration for our tagger. 

For next time, we will look deeper into the neural networks and use the tips Marcus sent us.

## Day 4, 14/4-2019
We prepared for tomorrows meeting by looking at Marcus tips that he sent us. Also we started looking at more journals and papers and found a really good one: [A Novel Approach for Protein-Named Entity Recognition and Protein-Protein Interaction Extraction](https://www.hindawi.com/journals/mpe/2015/942435/). They describe step-by-step a method with SVM to extract the entities. If they agree on tomorrows meeting we will follow this guide next week.

## Day 5, 15/4-2019
Today after the meeting we have a clearer understanding how to move forward. The process follows:
    1. Add list of corpus with summarization of each content and size to github
    2. [x] Fix java to ignore case
    3. [x] Complete a working model with the dictionary model
        i. [x] ~~Fix reader and use Element tree instead.~~ Also read in 'pmid' and change output to 'pmid \t abstract_text \n'.
        ii. [x] Match output to corpus annotation
    4. [ ] Create evaluator for dictionary model
        i. Use [genetag](https://www.ncbi.nlm.nih.gov/pubmed/15960837) for evaluation or chemner? or both?
        ii. Compare with e.g recall
    5. Add more to dictionary and run evaluator to see if it improves. If ambigueties use dominant write.
    
After this is completed we will start on the machine learning model.
    1. Use POS? Pre-processing?
    2. Sentence -> Tokens -> Tagger -> Phrases -> ML (BioLSTM and Bioascembedding(spelling?))
    3. Check out page [nlpprogress](http://nlpprogress.com/english/named_entity_recognition.html) for NER datasets
   

Möte med Sonja: https://biocreative.bioinformatics.udel.edu/resources/corpora/biocreative-iii-corpus/

---

In [ ]:
indx = app.loadIndex(get_java_file("protein_names_long.fst"))

data = ''
with open('genetag.db', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('\n>>')
dictionary = {}
for s in sp:
    lines = s.split('\n')
    #If it is an excerpt
    if lines[0] == 'EXCERPT':
        id = ''
        text = ''
        for line in lines[1:]:
            if line.startswith('EXCERPT_ID'):
                id = line.split(': ')[1]
            elif line.startswith('TEXT'):
                text = ''.join(line.split(': ')[1:])
        dictionary[id] = text

In [ ]:
# use indx.search to find all occurrences of proteins in abstracts from genetag. 
indx = app.loadIndex(get_java_file("protein_name_index.fst"))

d = dictionary
out = []
for key in d:
    value = d[key]
    hits = indx.search(value)    
    for hit in hits:
        # term.raw for term in hit.terms prints lists of protein names. Each list is protein name
        out.append('|'.join([key, str(hit.start) + ' ' + str(hit.end), " ".join([term.raw for term in hit.terms])]))
file = open('genetag.out', 'w+', encoding="utf-8", errors='ignore')
file.write('\n'.join(out))
file.close()
    

---
Today we sat for quite some time with merge conflicts in jupyter notebooks as it would not compile after a while. Also we tried to implement 2 models for evaluation; genetag with perl and another one with python 2.5. The gentag could have worked, but we got such a low score that it probably checks line by line simmilarity or something like it. This means we only got 8 correct matches which seems very low. We will look further into this next time.

We succesfully changed the tokenizers in java by removing 2 of them and adding one to ignore cases. We became aware of how many small words were in the 'shortName' tag from chemprot. So we got protein names like 'AM, OR, AND' etc. We thus created a textfile 'protein_names_long' which contains only the long names. This meant we got a more expected result, but we are also missing out on some of the real names. TODO: Think about how to include the short names - do we want to set a limit of min 2 characters?

Finally we created an out file to match the evaluation file in genetag. It reads from the database and using our proteinnames builds the same style file.

## Day 6, 17/4-2019
We started by creating another file 'protein_names_short', and discussed whether to use this in combination with long names or only long names. For now we will only use long.

Today's goal is to create/find/adapt an evaluator that works on our data.

We can run the perl evaluator from genetag and get the following result:
```
TP: 2
FP: 18211
FN: 406
Precision: 0.000109811672980838
Recall: 0.00490196078431373
```
Which is very bad. we think this is because the java tokenizer changes words like "h-55" to "h - 55". So it matches both 'h' and '55'. We need to change such that the hyphen does not get split in this way. There are probably more errors as well.


In [ ]:
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.algorithm import group_by_span, dominant_right_span, dominant_right
from docria.printout import options
set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    return jvm.java.io.File(os.path.abspath(path))

p_type = 'long'

#app.buildIndex(get_java_file("protein_name/protein_names_{}.txt".format(p_type)), get_java_file("protein_name/protein_names_{}.fst".format(p_type)))
#indx = app.loadIndex(get_java_file("protein_name/protein_names_{}.fst".format(p_type)))


#READ INPUT
data = ''
proteins = set()
with open('genetag.db', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('\n>>')
dictionary = {}

for s in sp:
    lines = s.split('\n')
    #If it is an excerpt
    if lines[0] == 'EXCERPT':
        id = ''
        text = ''
        for line in lines[1:]:
            if line.startswith('EXCERPT_ID'):
                id = line.split(': ')[1]
            elif line.startswith('TEXT'):
                text = ':'.join(line.split(':')[1:])[1:]
        dictionary[id] = text
    elif lines[0] == 'ANNOTATION':
        id = ''
        text = ''
        if 'ALTGENE' in lines[1]:
            continue
        for line in lines[2:]:
            if line.startswith('TEXT:'):
                text = ':'.join(line.split(':')[1:])[1:]
        proteins.add(text)
print(len(proteins))
with open('protein_name/protein_names_{}.txt'.format(p_type), 'a', encoding='utf-8', errors='ignore') as file:
    file.write('\n'.join(proteins))

app.buildIndex(get_java_file("protein_name/protein_names_{}.txt".format(p_type)), get_java_file("protein_name/protein_names_{}.fst".format(p_type)))
indx = app.loadIndex(get_java_file("protein_name/protein_names_{}.fst".format(p_type)))

In [ ]:
#import re
d = dictionary
out = []

for key in d:
    doc = Document()
    doc.add_text("main", d[key])    
    binary_doc = MsgpackCodec.encode(doc) 
    search_binary_doc = app.search(indx, binary_doc)
    doc = MsgpackCodec.decode(search_binary_doc)  

    tuls = []
    for term in doc["matches"]:
        tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
    print(tuls)    
    matches = [(x,y,z) for (x,y,z) in tuls if z in dominant_right(tuls)]
    
    for m in matches:
        #if re.search('(?!^\d+$)^.+$', m[2]): #removes pure numbers
            out.append('|'.join([key, str(m[0]) + ' ' + str(m[1]), m[2]]))
        
file = open('genetag/genetag_{}.out'.format(p_type), 'w+', encoding="utf-8", errors='ignore')
file.write('\n'.join(out))
file.close()

---
We chatted with Marcus for quite a bit today. As we wanted to add dominant_right we first tried to do it in Java but after a lot of trial-and-error it did not work and we switched to the python version. We also had to switch the way wee send in data as we needed to use docria instead of python.

Additionally we started getting back to the filters in java that we originally had. We also tried to work on the evaluator, but we are suspecting that something is weird in it as we get an unexpected nbr of matches (only ~27 TruePositive). 

For next time: add stopwords and fix min_length of matches.

### To run the perl evaluator
- go to medtag/genetag
- run `perl alt_eval.perl ../[PATH]/genetag_short.out Gold.format > answer.txt`


## Day 7 6/5-2019
Today's meeting: For next week we want a functioning demo. Want something to show eventhough it is not that accurate. GUI not priority but Pierre would like to do everything with the push of a button.

Keep 2 layers, one fuzzy and remove the matches from fuzzy if it overlaps with the exact layer.

TODO: Fix the Gold.format file. Right now it does not have the correct offset and thus gives us the terrible score. Second, we need to play around with the filters and keep building on 2 different analyzers for the short and long words. Finally tweak so that we can present a solution next time.

---

In [ ]:
#Fix gold format file
with open('../../medtag/genetag/Gold.format', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()

prot = {}
fixed = {} 
for line in data.split('\n'):
    l = line.split('|')
    if len(l) > 2:
        id = l[0]
        [i1, i2] = l[1].split()
        p = l[2]
        prot.setdefault(id, []).append(p)

for key in dictionary:
    if key in prot:
        proteins = prot[key]
        start_i = 0
        previous = ''
        for p in proteins:
            if p == previous:
                start_i += len(previous)
            match_start = start_i + dictionary[key][start_i:].index(p)
            match_end = match_start + len(p)
            start_i = match_start
            fixed.setdefault(key, []).append('{}|{} {}|{}'.format(key, match_start, match_end, p))
            previous = p

elems = []
for key in sorted(fixed.keys()):
    for e in fixed[key]:
        elems.append(e)

with open('Gold2.format', 'w+', encoding='utf-8', errors='ignore') as file:
    file.write('\n'.join(elems))


With this new Gold.format we got:

genetag_long: ```Precision: 0.517406105864265 Recall: 0.350740713169061```. Gives: F1 = 0.42

genetag_short: ```Precision: 0.11398451655411 Recall: 0.309527359475175```

We changed the filters for genetag short (only minimum length = 2 and split on whitespace) and recieved a better result:

genetag_short: ```Precision: 0.522418879056047 Recall: 0.291714709273596```. Gives: F1 = 0.37

Now we also need to find a way to combine genetag_short and long and score them together. One idea is to use dominant right for each abstract. Another is to simply smack them together. As the evaluator is not dependent on the order of the id's, we will try the simple solution of stacking them together first.

genetag: ```Precision: 0.288077969174977 Recall: 0.488497227255257```

When stacking them together we don't get TP(short)+TP(long), so there seems to be an overlap of ~4000 proteins. So we must use something like dominant right to fix this overlap.

In [ ]:
#Build together long and short


## Day 8 7/5-2019
We have consulted Marcus on what todo going forward with adding genetag_short and genetag_long. He said we have 2 options:
    1. Add them both to one layer and do Dominant Right
    2. Prioritize one detection method over the other, i.e. find all the intersections, remove the overlaps and then join the layers.

So for today, our todo is as follows:
    1. Remove numbers from the genetag lists (pure numbers like 555)
    2. Join genetag long and short using one of the methods above
    3. Evaluate and find possible improvements
    4. Add list of cell death synonyms
    5. Fix demo that we can show
    
---

We will first look at the filters in java to see if there is one ready to use. There wasn't one. We will then do some postprocessing and remove those matches. We did this ```if re.search('(?!^\d+$)^.+$', m[2]): #removes pure numbers``` which will suffice for now.

genetag_short: ```Precision: 0.523598384077249 Recall: 0.291769615110086``` Without the only number ones. So slightly improved (by ~3 decimals).

genetag_long: ```Precision: 0.336532053845129 Recall: 0.415911711414923``` So this got worse recall? What happened? 

(genetag_long: ```Precision: 0.336532053845129 Recall: 0.415911711414923``` F1 = 0.37. When we swiched ICUFoldingFilter to toLowerCaseFilter. So we'll go with ICU instead)

In [ ]:
#Remove numbers from the genetag lists

Trying different filters for genetag_long:

| Activated filter | Precision | Recall | F1 |
|----------------------------------------|-----------|--------|----|
| Mention, ICUNorm, Diacratic, Snowball  | 0.623715592889822 | 0.456596936254324 | 0.527 |
| Mention, ICUNorm, Diacratic, Snowball, ICUFolding | 0.350802099229538 | 0.517487508922198 | 0.418 |
| Mention, ICUNorm, Diacratic, Snowball, StopFilter | 0.615680473372781 | 0.457036182946247 | 0.524 |
| Mention, ICUNorm, Diacratic, Snowball, LengthFilter | 0.287200716043858 | 0.422829846812716 | 0.342 |
| Mention, ICUNorm, Diacratic, Snowball, toLowerCase | 0.350802099229538 | 0.517487508922198 | 0.418 |
| Mention, ICUNorm, Diacratic | 0.651159123166693 | 0.45341239773788 | 0.535 |
| Mention, ICUNorm |  0.651159123166693 | 0.45341239773788 | 0.535 |
| Mention |  0.651159123166693 | 0.45341239773788 | 0.535 |
| Mention (without removing numbers) |  0.651107782070488 | 0.45341239773788 | 0.535 |

So for genetag_long the best solution seemed to be to not really add anything.

---

Different filters for genetag_short:

| Activated filter | Precision | Recall | F1 |
|----------------------------------------|-----------|--------|----|
| PatternTokenizerFactory | 0.61393618220769 | 0.298962279690331 | 0.402 |
| Mention | 0.629489888318744 | 0.458024488003075 | 0.530 |
| Mention, LengthFilter | 0.324438202247191 | 0.418547191566464 | 0.366 |
| Mention (without removing numbers) | 0.629489888318744 | 0.458024488003075 | 0.530 |


In [ ]:
#Join genetag long and short using one of the methods above
data_long = ''
data_short = ''

out = []

with open('genetag/genetag_long.out', 'r', encoding='utf-8', errors='ignore') as file:
    data_long = file.read().split('\n')
with open('genetag/genetag_short.out', 'r', encoding='utf-8', errors='ignore') as file:
    data_short = file.read().split('\n')

def read(data, dic):
    for line in data:
        tul = line.split('|')
        key = tul[0]
        line_nbr = tul[1].split()
        start = line_nbr[0]
        end = line_nbr[1]
        word = tul[2]
        dic.setdefault(key, []).append((int(start), int(end), word))
    return dic

dic = read(data_short, read(data_long, {}))

for key in dic:
    tuls = dic[key]
    matches = [(x,y,z) for (x,y,z) in tuls if z in dominant_right(tuls)]
    matches = list(set(matches))
    
    for m in matches:
        out.append('|'.join([key, str(m[0]) + ' ' + str(m[1]), m[2]]))
        
with open('genetag/genetag.out', 'w+', encoding="utf-8", errors='ignore') as file:
    file.write('\n'.join(out))

Combined genetag_long and genetag_short, i.e. genetag.out: ```0.592672838290329 Recall: 0.495634985999012```
F1 = 0.540

In [ ]:
# Prototype
text = 'Peroxydase reaction stains were negative, chloroacetate esterase were strongly positive.'
file_name = 'protein_name/protein_names_long'


def get_proteins(s, f):
    from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
    import os.path
    from docria.printout import options
    from docria.algorithm import group_by_span, dominant_right_span, dominant_right
    set_large_screen()
    def connect_jvm(port):
        from py4j.java_gateway import GatewayParameters, JavaGateway
        gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
        gateway = JavaGateway(gateway_parameters=gateway_parameters)
        app = gateway.entry_point
        return gateway, gateway.jvm, app
    gateway, jvm, app = connect_jvm(6006)
    def get_java_file(path):
        # Here we do the py4j equivalent for new java.io.File(path)
        return jvm.java.io.File(os.path.abspath(path))
    app.buildIndex(get_java_file("{}.txt".format(f)), get_java_file("{}.fst".format(f)))
    indx = app.loadIndex(get_java_file("{}.fst".format(f)))
    doc = Document()
    doc.add_text("main", s)
    binary_doc = MsgpackCodec.encode(doc) 
    search_binary_doc = app.search(indx, binary_doc)
    doc = MsgpackCodec.decode(search_binary_doc)
    tuls = []
    for term in doc["matches"]:
        tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
        #print(str(term["text"]), "-", "%d:%d" % (term["text"].start, term["text"].stop))
    print(tuls)
    return dominant_right(tuls)

matches = get_proteins(text, file_name)
print('--------')
print(matches)

In [ ]:
#Do everything for both short and long names
name = 'protein_name/protein_names_all';

from xml.etree import ElementTree as ET
filename='uniprot_sprot.xml'
parser = ET.iterparse(filename)
length = 28 #length of header in xml tags
proteins = set()
with open('{}.txt'.format(name), 'w') as f:
    for event, element in parser:
        tag = element.tag[length:]
        if 'shortName' == tag or 'fullName' == tag:
            proteins.add(element.text)  
        element.clear()
    f.write('\n'.join(proteins))

In [ ]:
############################################################################
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.algorithm import group_by_span, dominant_right_span, dominant_right
from docria.printout import options
set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    return jvm.java.io.File(os.path.abspath(path))
############################################################################

app.buildIndex(get_java_file('{}.txt'.format(name)), get_java_file('{}.fst'.format(name)))
indx = app.loadIndex(get_java_file('{}.fst'.format(name)))


#READ INPUT
data = ''
proteins = set()
with open('genetag.db', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('\n>>')
dictionary = {}

for s in sp:
    lines = s.split('\n')
    #If it is an excerpt
    if lines[0] == 'EXCERPT':
        id = ''
        text = ''
        for line in lines[1:]:
            if line.startswith('EXCERPT_ID'):
                id = line.split(': ')[1]
            elif line.startswith('TEXT'):
                text = ':'.join(line.split(':')[1:])[1:]
        dictionary[id] = text
    elif lines[0] == 'ANNOTATION':
        id = ''
        text = ''
        for line in lines[1:]:
            if line.startswith('TEXT:'):
                text = ':'.join(line.split(':')[1:])[1:]
        proteins.add(text)

with open('{}.txt'.format(name), 'a', encoding='utf-8', errors='ignore') as file:
    file.write('\n'.join(proteins))
file.close()

app.buildIndex(get_java_file('{}.txt'.format(name)), get_java_file('{}.fst'.format(name)))
indx = app.loadIndex(get_java_file('{}.fst'.format(name)))
############################################################################
#import re
d = dictionary
out = []

for key in d:
    doc = Document()
    doc.add_text("main", d[key])    
    binary_doc = MsgpackCodec.encode(doc) 
    search_binary_doc = app.search(indx, binary_doc)
    doc = MsgpackCodec.decode(search_binary_doc)  

    tuls = []
    for term in doc["matches"]:
        tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
    matches = [(x,y,z) for (x,y,z) in tuls if z in dominant_right(tuls)]
    
    for m in matches:
        #if re.search('(?!^\d+$)^.+$', m[2]): #removes pure numbers
            out.append('|'.join([key, str(m[0]) + ' ' + str(m[1]), m[2]]))
        
with open('genetag/genetag_all.out'.format(p_type), 'w+', encoding="utf-8", errors='ignore') as file:
    file.write('\n'.join(out))

The method above did not work

## Day 9, 9/5-2019
From last time we still had the following tasks in our todo list left:
    3. Evaluate and find possible improvements
    4. Add list of cell death synonyms
    5. Fix demo that we can show (!)
    
However, last time we ran into a problem with the evaluator (again!). It seems that the Gold.format does not include all the bits that is included in the annotation bits. Some of those seem to be included in Correct.format. What is the difference in these two? Where is the bug?

We started by adding all the code we need below to prepare for the prototyping. Also, we ran the tests again to make sure that this did not change the outcome we got previously.

Only do the following snippets once:

In [ ]:
#Goes through the file uniprot_sprot.xml and saves the names of the proteins
#in protein_names.txt
name = 'short'

from xml.etree import ElementTree as ET
filename='uniprot_sprot.xml'
parser = ET.iterparse(filename)
length = 28 #length of header in xml tags
proteins = set()
for event, element in parser:
    tag = element.tag[length:]
    if 'shortName' in tag: # or 'shortName' in tag #fullName
        proteins.add(element.text)
    element.clear()
with open('protein_name/protein_names_{}.txt'.format(name), 'w+') as f:
    f.write('\n'.join(proteins))

In [ ]:
#Goes through the file uniprot_sprot.xml and saves the names of the proteins
#in protein_names.txt
name = 'long_no_anno'

from xml.etree import ElementTree as ET
filename='uniprot_sprot.xml'
parser = ET.iterparse(filename)
length = 28 #length of header in xml tags
proteins = set()
for event, element in parser:
    tag = element.tag[length:]
    if 'fullName' == tag: # or 'shortName' in tag #fullName
        proteins.add(element.text)
    element.clear()
    
with open('protein_name/protein_names_{}.txt'.format(name), 'w+') as f:
    f.write('\n'.join(proteins))

#READ INPUT
data = ''
proteins = set()
with open('genetag.db', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
    
sp = data.split('\n>>')
dictionary = {}

for s in sp:
    lines = s.split('\n')
    if lines[0] == 'ANNOTATION' and 'ALTGENE' not in s:
        id = ''
        text = ''
        for line in lines[1:]:
            if line.startswith('TEXT:'):
                text = ':'.join(line.split(':')[1:])[1:]
        proteins.add(text)

with open('protein_name/protein_names_{}.txt'.format(name), 'a', encoding='utf-8', errors='ignore') as file:
    file.write('\n'.join(proteins))


In [ ]:
#READ ABSTRACTS TO FILE
import json

data = ''

with open('genetag.db', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
    
sp = data.split('\n>>')
dictionary = {}

for s in sp:
    lines = s.split('\n')
    if lines[0] == 'EXCERPT':
        id = ''
        text = ''
        for line in lines[1:]:
            if line.startswith('EXCERPT_ID'):
                id = line.split(': ')[1]
            elif line.startswith('TEXT'):
                text = ':'.join(line.split(':')[1:])[1:]
        dictionary[id] = text

with open('genetag/abstracts_genetag.txt', 'w+',encoding="utf-8", errors='ignore') as file:
    file.write(json.dumps(dictionary))

Below is to be run everytime there is a change in the Java code (i.e changed filters) or you want to switch between long/short words.  

In [ ]:
import json

p_type = 'uniprot'
c_type = 'cell_death_names'

from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.algorithm import group_by_span, dominant_right_span, dominant_right
from docria.printout import options
set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    return jvm.java.io.File(os.path.abspath(path))

app.buildIndex(get_java_file("protein_name/protein_names_{}.txt".format(p_type)), get_java_file("protein_name/protein_names_{}.fst".format(p_type)))
indx = app.loadIndex(get_java_file("protein_name/protein_names_{}.fst".format(p_type)))

app.buildIndex(get_java_file("{}.txt".format(p_type)), get_java_file("{}.fst".format(p_type)))
indx = app.loadIndex(get_java_file("protein_name/protein_names_{}.fst".format(p_type)))

d = {}

with open('genetag/abstracts_genetag.txt', 'r',encoding="utf-8", errors='ignore') as file:
    d = json.loads(file.read())

    
# Write to genetag
out = []

for key in d:
    doc = Document()
    doc.add_text("main", d[key])    
    binary_doc = MsgpackCodec.encode(doc) 
    search_binary_doc = app.search(indx, binary_doc)
    doc = MsgpackCodec.decode(search_binary_doc)  

    tuls = []
    for term in doc["protmatches"]:
        tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
    matches = tuls #[(x,y,z) for (x,y,z) in tuls if z in dominant_right(tuls)]
    
    for m in matches:
        #if re.search('(?!^\d+$)^.+$', m[2]): #removes pure numbers
        out.append('|'.join([key, str(m[0]) + ' ' + str(m[1]), m[2]]))
        
with open('genetag/genetag_{}.out'.format(p_type), 'w+', encoding="utf-8", errors='ignore') as file:
    file.write('\n'.join(out))


In [ ]:
#Join genetag long and short
data_long = ''
data_short = ''

out = []

with open('genetag/genetag_long.out', 'r', encoding='utf-8', errors='ignore') as file:
    data_long = file.read().split('\n')
with open('genetag/genetag_short.out', 'r', encoding='utf-8', errors='ignore') as file:
    data_short = file.read().split('\n')

def read(data, dic):
    for line in data:
        tul = line.split('|')
        key = tul[0]
        line_nbr = tul[1].split()
        start = line_nbr[0]
        end = line_nbr[1]
        word = tul[2]
        dic.setdefault(key, []).append((int(start), int(end), word))
    return dic

dic = read(data_short, read(data_long, {}))

for key in dic:
    tuls = dic[key]
    matches = [(x,y,z) for (x,y,z) in tuls if z in dominant_right(tuls)]
    matches = list(set(matches))
    
    for m in matches:
        out.append('|'.join([key, str(m[0]) + ' ' + str(m[1]), m[2]]))
        
with open('genetag/genetag.out', 'w+', encoding="utf-8", errors='ignore') as file:
    file.write('\n'.join(out))

Prototype code and to use to test for bugs:

In [ ]:
# Prototype
text = "The MB-CPK isoenzyme showed no percentage increase of total CPK higher than 5%, measured at 6, 12, and 24 h after the shock, independent of the number of attempts of cardioversion."
file_name = 'protein_name/protein_names_long'


def get_proteins(s, f):
    from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
    import os.path
    from docria.printout import options
    from docria.algorithm import group_by_span, dominant_right_span, dominant_right
    set_large_screen()
    def connect_jvm(port):
        from py4j.java_gateway import GatewayParameters, JavaGateway
        gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
        gateway = JavaGateway(gateway_parameters=gateway_parameters)
        app = gateway.entry_point
        return gateway, gateway.jvm, app
    gateway, jvm, app = connect_jvm(6006)
    def get_java_file(path):
        # Here we do the py4j equivalent for new java.io.File(path)
        return jvm.java.io.File(os.path.abspath(path))
    app.buildIndex(get_java_file("{}.txt".format(f)), get_java_file("{}.fst".format(f)))
    indx = app.loadIndex(get_java_file("{}.fst".format(f)))
    doc = Document()
    doc.add_text("main", s)
    binary_doc = MsgpackCodec.encode(doc) 
    search_binary_doc = app.search(indx, binary_doc)
    doc = MsgpackCodec.decode(search_binary_doc)
    tuls = []
    for term in doc["matches"]:
        tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
        #print(str(term["text"]), "-", "%d:%d" % (term["text"].start, term["text"].stop))
    print(tuls)
    return dominant_right(tuls)

matches = get_proteins(text, file_name)
print('--------')
print(matches)

### New score after all of this:
* TP: 8462
* FP: 5066
* FN: 9751
* Precision: 0.62551744529864 Recall: 0.464613188381925
* F1:0.533

In [ ]:
#Fix correct format file
with open('../../medtag/genetag/Correct.data', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()

#READ INPUT
d = ''
proteins = set()
with open('genetag.db', 'r', encoding="utf-8", errors='ignore') as file:
    d = file.read()
    
sp = d.split('\n>>')
dictionary = {}

for s in sp:
    lines = s.split('\n')
    if lines[0] == 'EXCERPT':
        id = ''
        text = ''
        for line in lines[1:]:
            if line.startswith('EXCERPT_ID:'):
                id = line.split(': ')[1]
            elif line.startswith('TEXT:'):
                text = ':'.join(line.split(':')[1:])[1:]
        dictionary[id] = text
        
prot = {}
fixed = {} 

for line in data.split('\n'):
    l = line.split('|')
    if len(l) > 2:
        id = l[0]
        [i1, i2] = l[1].split()
        p = l[2]
        prot.setdefault(id, []).append(p)

for key in dictionary:
    if key in prot:
        proteins = prot[key]
        start_i = 0
        previous = ''
        for p in proteins:
            if p == previous:
                start_i += 0
            if p in dictionary[key][start_i:]:
                match_start = start_i + dictionary[key][start_i:].index(p)
                match_end = match_start + len(p)
                start_i = match_start
                fixed.setdefault(key, []).append('{}|{} {}|{}'.format(key, match_start, match_end, p))
                previous = p
            else:
                print(previous)
                print(p)
                print(dictionary[key])
                print(dictionary[key][start_i:])
                print('------------------------')

elems = []
for key in sorted(fixed.keys()):
    for e in fixed[key]:
        elems.append(e)

with open('Correct2.data', 'w+', encoding='utf-8', errors='ignore') as file:
    file.write('\n'.join(elems))


We found a bug! When adding things to our dictionary we also add the 'ALTGENE', which is not included in the Gold.format file (it is in the Correct.data file). Thus things like this gets us in trouble:

* ALTGENE: vesicular stomatitis virus glycoprotein (VSVG)
* GENE: vesicular stomatitis virus glycoprotein
* GENE: VSVG

Because of dominant right we pick the ALTGENE as the correct one which Gold.format gives us the wrong answer for. Fixing that gives:

* TP: 11046
* FP: 2015
* FN: 7167
* Precision: 0.845723910879718 Recall: 0.606489869873168
* F1 = 0.706

---

We found another bug:

In the evaluation file:
- FN|P07691837A1359|36 51|K3 keratin gene
- TP|P07691837A1359|NFkB|185 189|NFkB|185 189

In protein_names_long:
- K3 keratin gene

In genetag_long (nothing in genetag_short):
- P07691837A1359|185 189|NFkB

TEXT: 
- This 300 bp 5'-upstream sequence of K3 keratin gene, which can function in vitro as a keratinocyte-specific promoter, contains two clusters of partially overlapping motifs, one with an NFkB consensus sequence and another with a GC box.

---
Another bug:
If we have the text
- 'The **MB-CPK** isoenzyme showed no percentage increase of total CPK higher than 5%, measured at 6, 12, and 24 h after the shock, independent of the number of attempts of cardioversion.' 

We find CPK, but not MB. However if they switch places, i.e **'CPK-MB'**, then we only find MB and not CPK.


## Day 10, 13/5-2019

Today we need to solve the bugs from yesterday and also deliver a demo for tomorrow's meeting. Marcus told us to run the text through the analyzer pipeline and show the output to see if all the tokens are there. The MentionTokenizer might have some bugs, so we might to to switch that aswell.

This is the output from the terms in 'The MB-CPK isoenzyme':

```
  field    value                   

  term     'MB'                    
  text     span(main[4:6]) = 'MB'  
  type     'WORD_ACRONYM'          

           Node terms#2           

  field    value                  

  term     '-'                    
  text     span(main[6:7]) = '-'  
  type     'HYPHEN'               

            Node terms#3             

  field    value                     

  term     'CPK'                     
  text     span(main[7:10]) = 'CPK'  
  type     'WORD_ACRONYM'            

                Node terms#4                
``` 

And still we only get the match: [(7, 10, 'CPK')]

---

We found the bug! It was because of the dominant right we implemented in Java. We forgot about that since we have one in the pyhon code aswell.        //matches.retainAll(DominantRight.resolve(matches, "text"));

Now we have the following score for genetag_long:

```
TP: 17090
FP: 4845
FN: 1123
Precision: 0.779120127649875 Recall: 0.93834074562126
```

But genetag_short only gives:
```
TP: 2066
FP: 4610
FN: 16147
Precision: 0.309466746554823 Recall: 0.113435458189206
```

---

Apperantly we also need to divide the words from the annotaded part in genetag into training and test-set.

genetag_long without the words in the annotations:
```
TP: 753
FP: 2010
FN: 17460
Precision: 0.272529858849077 Recall: 0.0413440948772855
```

In [ ]:
# Divide Gold into test and train
div = 0.8

gold = ''
with open('Gold2.format', 'r') as f:
    gold = f.read()

gold = gold.split('\n')

train = round(len(gold)*div)

train_gold = gold[:train]
test_gold = gold[train:]

with open('Gold2_train.format', 'w+', encoding='utf-8') as f:
    f.write('\n'.join(train_gold))
    
id = set()
for g in test_gold:
    id.add(g.split('|')[0])

with open('Gold2_test.format', 'w+', encoding='utf-8') as f:
    f.write('\n'.join(test_gold))
    
with open('Gold2_test_ids.out', 'w+', encoding='utf-8') as f:
    f.write('\n'.join(id))

In [ ]:
# Build prot_names based on test and train
name = 'long_no_anno'

from xml.etree import ElementTree as ET
filename='uniprot_sprot.xml'
parser = ET.iterparse(filename)
length = 28 #length of header in xml tags
proteins = set()
for event, element in parser:
    tag = element.tag[length:]
    if 'fullName' == tag: # or 'shortName' in tag #fullName
        proteins.add(element.text)
    element.clear()
    
with open('protein_name/protein_names_{}.txt'.format(name), 'w+') as f:
    f.write('\n'.join(proteins))

p_id_test = []
with open('Gold2_test_ids.out', 'r') as f:
    p_id_test = f.read().split('\n')

#READ INPUT
data = ''
proteins = set()
with open('genetag.db', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
    
sp = data.split('\n>>')
dictionary = {}

for s in sp:
    lines = s.split('\n')
    if lines[0] == 'ANNOTATION' and 'ALTGENE' not in s:
        text = ''
        for line in lines[1:]:
            if line.startswith('EXCERPT_ID:') and line.split('ID: ')[1] in p_id_test:
                break
            if line.startswith('TEXT:'):
                text = ':'.join(line.split(':')[1:])[1:]
        proteins.add(text)

with open('protein_name/protein_names_{}.txt'.format(name), 'a', encoding='utf-8', errors='ignore') as file:
    file.write('\n'.join(proteins))

In [ ]:
#Build genetag with prot names train

import json

p_type = 'uniprot'

from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.algorithm import group_by_span, dominant_right_span, dominant_right
from docria.printout import options
set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    return jvm.java.io.File(os.path.abspath(path))

app.buildIndex(get_java_file("protein_name/protein_names_{}.txt".format(p_type)), get_java_file("protein_name/protein_names_{}.fst".format(p_type)))
indx = app.loadIndex(get_java_file("protein_name/protein_names_{}.fst".format(p_type)))

d = {}
with open('genetag/abstracts_genetag.txt', 'r',encoding="utf-8", errors='ignore') as file:
    d = json.loads(file.read())

p_id_test = []
with open('Gold2_test_ids.out', 'r') as f:
    p_id_test = f.read().split('\n')
    
# Write to genetag
out = []

for key in d:
    if key in p_id_test:
        doc = Document()
        doc.add_text("main", d[key])    
        binary_doc = MsgpackCodec.encode(doc) 
        search_binary_doc = app.search(indx, binary_doc)
        doc = MsgpackCodec.decode(search_binary_doc)  

        tuls = []
        for term in doc["matches"]:
            tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
        matches = [(x,y,z) for (x,y,z) in tuls if z in dominant_right(tuls)]

        for m in matches:
            #if re.search('(?!^\d+$)^.+$', m[2]): #removes pure numbers
            out.append('|'.join([key, str(m[0]) + ' ' + str(m[1]), m[2]]))
        
with open('genetag/genetag_{}.out'.format(p_type), 'w+', encoding="utf-8", errors='ignore') as file:
    file.write('\n'.join(out))

På testset får vi då (med long bara):
```
TP: 1293
FP: 1079
FN: 2350
Precision: 0.545109612141653 Recall: 0.354927257754598
```
På train får vi:
```
TP: 13744
FP: 4101
FN: 826
Precision: 0.770187727654805 Recall: 0.943308167467399
```

In [ ]:
# Prototype

import re

def get_cell(text, f):
    s_out = text
    synonyms = []
    with open(f, 'r') as f:
        synonyms = f.read().split('\n')
    
    for s in synonyms:
        if s.lower() in text.lower():
            s_out = re.sub(s, "\033[45m{}\033[m".format(s), s_out, flags = re.IGNORECASE)            
    return s_out

def get_proteins(s, f):
    from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
    import os.path
    from docria.printout import options
    from docria.algorithm import group_by_span, dominant_right_span, dominant_right
    set_large_screen()
    def connect_jvm(port):
        from py4j.java_gateway import GatewayParameters, JavaGateway
        gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
        gateway = JavaGateway(gateway_parameters=gateway_parameters)
        app = gateway.entry_point
        return gateway, gateway.jvm, app
    gateway, jvm, app = connect_jvm(6006)
    def get_java_file(path):
        # Here we do the py4j equivalent for new java.io.File(path)
        return jvm.java.io.File(os.path.abspath(path))
    app.buildIndex(get_java_file("{}.txt".format(f)), get_java_file("{}.fst".format(f)))
    indx = app.loadIndex(get_java_file("{}.fst".format(f)))
    doc = Document()
    doc.add_text("main", s)
    binary_doc = MsgpackCodec.encode(doc) 
    search_binary_doc = app.search(indx, binary_doc)
    doc = MsgpackCodec.decode(search_binary_doc)
    tuls = []
    for term in doc["matches"]:
        tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
        #print(str(term["text"]), "-", "%d:%d" % (term["text"].start, term["text"].stop))
    tul = [x[2] for x in tuls]

    dr = dominant_right(tuls)
    #dr = tul # without dominant_right --> primase
    
    i = 0; j=0; s_out = ''

    for d in dr:
        while j < len(tuls):
            j+=1
            if d == tuls[j-1][2]:
                print('MATCH:')
                print(d)
                print('------------------')
                i_s = tuls[j-1][0]
                i_e = tuls[j-1][1]

                s_out += s[i:i_s] + "\033[43m{}\033[m".format(s[i_s:i_e])
                i = i_e
                break

    return s_out + s[i:]


# '\033[43m{}\033[m.format'
#text = "The MB-CPK isoenzyme showed no percentage increase of total CPK higher than 5%, measured at 6, 12, and 24 h after the shock, independent of the number of attempts of cardioversion. K3 keratin gene"
text = "Apoptosis. The constraints of primase recognition sequences, nucleotide substrate requirements, and the effects of additional proteins on oligoribonucleotide synthesis by the 63-kDa gene 4 protein have been examined using templates of defined sequence. cell death is caused by cells dying."

file_name = 'protein_name/protein_names_long'

matches = "{} \n\n\033[43m{}\033[m\n\033[45m{}\033[m".format(get_cell(get_proteins(text, file_name), 'cell_death_names.txt'), 'PROTEIN', 'CELL DEATH')
print(matches)

### Summary
So today we finished a prototype that can highlight words in the terminal (probably won't work if we write to file), fixed the bugs by removing dominant right in java, and finally, splitting the annotated words into test/train data to use for the genetag which gave a worse result than before (as expected

## Day 11, 14/5-2019

Meeting w. Sonja and Pierre:
* Collect all the names (even the alternative ones, use gene name as key, in some of them there are multiple gene ones - but not in the text file??), USE THE UNIPROT IDENTIFIER INSTEAD AS THE KEY!
* Use spart of speach tagger to find what might be protein names and to remove unnecessary bits.
* Use spacey
* Apply this to a part of PUBMED (100 000- 200 000 nbr of abstracts) + benchmark the time it takes

* Pro vs euc, 'taxanomic lineage', if it is pro we do not use those entries

In [ ]:
#Read the new uniprot file and store them in protein names
filename='uniprot_less.txt'
data = []
with open(filename, 'r') as f:
    data = f.read().split('//')[:-1] #remove last elem (only whitespace)

dic = {}
prots = []
ids = []

for entry in data:
    lines = entry.split('\n')
    if 'OC   Eukaryota' not in entry:
        continue
    id = lines[1].split()[1]
    prot = []
    
    for line in lines[2:]:
        if line.startswith('DE') or line.startswith('GN'):
            p = re.search('(?<!C)=.+?( {|;)', line)
            if p:
                prot.append(p.group()[1:-1])
                prots.append(p.group()[1:-1])
                ids.append(id)
                
    dic[id] = prot  
    
with open('protein_name/protein_names_uniprot.txt', 'w+') as f:
    f.write('\n'.join(prots))
    
with open('protein_name/protein_names_uniprot_indx.txt', 'w+') as f:
    f.write('\n'.join(ids))

## Day 12, 16/5 - 2019


In [ ]:
#Print to file using docria and fix output for Vilhelm and Olof

import re
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.printout import options
from docria.algorithm import group_by_span, dominant_right_span, dominant_right
###
from docria.storage import DocumentIO

def get_proteins(s, f):
    set_large_screen()
    def connect_jvm(port):
        from py4j.java_gateway import GatewayParameters, JavaGateway
        gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
        gateway = JavaGateway(gateway_parameters=gateway_parameters)
        app = gateway.entry_point
        return gateway, gateway.jvm, app
    gateway, jvm, app = connect_jvm(6006)
    def get_java_file(path):
        return jvm.java.io.File(os.path.abspath(path))
    app.buildIndex(get_java_file("{}.txt".format(f)), get_java_file("{}.fst".format(f)))
    app.buildIndex(get_java_file("cell_death_names.txt"), get_java_file("cell_death_names.fst"))

    
    indx = app.loadIndex(get_java_file("{}.fst".format(f)))
    indx2 = app.loadIndex(get_java_file("cell_death_names.fst"))
    
    doc = Document()
    doc.add_text("main", s)   
    
    binary_doc = MsgpackCodec.encode(doc) 
    search_binary_doc = app.search(indx, indx2, binary_doc)

    doc = MsgpackCodec.decode(search_binary_doc)  
    
    tuls = []
    for term in doc["protmatches"]:
        tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
    
    tuls2 = []
    for term in doc["lysomatches"]:
        tuls2.append((term["text"].start, term["text"].stop, str(term["text"])))
    
    dr = dominant_right(tuls)
    
    i = 0; j=0; s_out = ''

    for d in dr:
        while j < len(tuls):
            j+=1
            if d == tuls[j-1][2]:
                print('MATCH, PROT:')
                print(d)
                print('------------------')
                i_s = tuls[j-1][0]
                i_e = tuls[j-1][1]

                s_out += s[i:i_s] + "\033[43m{}\033[m".format(s[i_s:i_e])
                i = i_e
                break
    s_out += s[i:]

    for d in tuls2:
        print('MATCH, LYSO: ')
        print(d[2])
        print('------------------')
        s_out = re.sub(d[2], "\033[45m{}\033[m".format(d[2]), s_out, flags = re.IGNORECASE) 

    with DocumentIO.write('output-file.docria') as dw:
        dw.write(doc)
    #with DocumentIO.read('output-file.docria') as dr:
    #    for doc in dr:
    #        print(doc["protmatches"])  
    #        print(doc["lysomatches"])  
                
    return s_out


# '\033[43m{}\033[m.format'
#text = "The MB-CPK isoenzyme showed no percentage increase of total CPK higher than 5%, measured at 6, 12, and 24 h after the shock, independent of the number of attempts of cardioversion. K3 keratin gene"
text = "apoptosis. The constraints of Cr10HGO, 110 kDa antigen, and the effects of additional proteins on oligoribonucleotide synthesis by the 63-kDa gene 4 protein have been examined using templates of defined sequence. cell death is caused by cells dying."

file_name = 'protein_name/protein_names_uniprot'

matches = "{} \n\n\033[43m{}\033[m\n\033[45m{}\033[m".format(get_proteins(text, file_name), 'PROTEIN', 'CELL DEATH')
print(matches)

In [ ]:
import json

p_type = 'uniprot(2)'
c_type = 'cell_death_names'

from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.algorithm import group_by_span, dominant_right_span, dominant_right
from docria.printout import options
from docria.storage import DocumentIO

set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    return jvm.java.io.File(os.path.abspath(path))

app.buildIndex(get_java_file("protein_name/protein_names_{}.txt".format(p_type)), get_java_file("protein_name/protein_names_{}.fst".format(p_type)))
indx = app.loadIndex(get_java_file("protein_name/protein_names_{}.fst".format(p_type)))

app.buildIndex(get_java_file("{}.txt".format(c_type)), get_java_file("{}.fst".format(c_type)))
indx2 = app.loadIndex(get_java_file("{}.fst".format(c_type)))

d = {}

with open('genetag/abstracts_genetag.txt', 'r',encoding="utf-8", errors='ignore') as file:
    d = json.loads(file.read())

    
# Write to genetag
out = []
dr = []
for key in d:
    doc = Document()
    doc.add_text("main", d[key])  
    doc.props["id"] = key 
    binary_doc = MsgpackCodec.encode(doc) 
    search_binary_doc = app.search(indx, indx2, binary_doc)
    doc = MsgpackCodec.decode(search_binary_doc)  

    dr.append(doc)

i = 0
j = 0
while i < len(dr):
    with DocumentIO.write('pubmed/pubmed1905({}).docria'.format(str(j))) as dw:
        while i < len(dr) and i < 50000:
            dw.write(dr[i])
            i += 1
    j += 1

    #for m in matches:
        #if re.search('(?!^\d+$)^.+$', m[2]): #removes pure numbers
    #    out.append('|'.join([key, str(m[0]) + ' ' + str(m[1]), m[2]]))
        
#with open('genetag/genetag_{}.out'.format(p_type), 'w+', encoding="utf-8", errors='ignore') as file:
#    file.write('\n'.join(out))


In [ ]:
#Turn docra file into readable temp file
with DocumentIO.read('docria/out_json_pubmed19n0001.xml.txt.docria') as dr:
    with open('docria/temp.txt', 'w+', encoding="utf-8", errors='ignore') as f:
        for doc in dr:
            f.write(str(doc.text["main"]))
            f.write(str(doc.props))
            for d in doc["protmatches"]:
                f.write(str(d))  
            for d in doc["lysomatches"]:
                f.write(str(d))
            f.write("--------------------\n\n")
          

In [ ]:
#Append genetag names to protein names
name = 'uniprot'

prot_names = ''
with open('protein_name/protein_names_{}.txt'.format(name), 'r') as f:
    prot_names = f.read()

#READ INPUT
data = ''
proteins = set()
with open('genetag.db', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
    
for s in data.split('\n>>'):
    lines = s.split('\n')
    if lines[0] == 'ANNOTATION' and 'ALTGENE' not in s:
        text = ''
        for line in lines[1:]:
            if line.startswith('TEXT:'):
                text = ':'.join(line.split(':')[1:])[1:]
        proteins.add(text)

with open('protein_name/protein_names_{}(2).txt'.format(name), 'w+', encoding='utf-8', errors='ignore') as file:
    file.write(prot_names)
    file.write('\n'.join(proteins))

In [ ]:
from xml.etree import ElementTree as ET
import json

filename='pubmed19n0651.xml'
parser = ET.iterparse(filename)

id = ''
abbe = {}
for event, element in parser:
    tag = element.tag
    if tag == 'PMID':
        id = element.text
    elif tag == 'AbstractText':
        abbe.setdefault(id, []).append(element.text)
    element.clear()

dictionary = {}

for key in abbe:
    to_add = abbe[key]
    if abbe[key] is str and len(abbe[key]) > 1:
        to_add = ' '.join(abbe[key]) 
    dictionary[key] = to_add

with open('pubmed/abstracts19n0651.txt', 'w+', encoding="utf-8", errors='ignore') as file:
    for key in abbe:
        file.write(key)
        file.write(str(abbe[key]))
        
with open('pubmed/abstracts19n0651_json.txt', 'w+',encoding="utf-8", errors='ignore') as file:
    file.write(json.dumps(dictionary))

## Day 13, 17/5
Todays goals:
1. Benchmark time it takes for pubmed CHECK!
2. Experiment with spacey +  Try a part of speech tagger to improve findings CHECK!
3. Find a way to remove noise, how much should we remove? CHECK!
4. A method that takes a protein name and outputs it's genetag id? Or add it to docria?
5. Clean up code CHECK!

In [ ]:
#Process protein names and cell death names using docria and print docria to file

import json
import re
import time
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.algorithm import group_by_span, dominant_right_span, dominant_right
from docria.printout import options
from docria.storage import DocumentIO
from nltk.corpus import stopwords


set_large_screen()

def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app

def get_java_file(jvm, path):
        return jvm.java.io.File(os.path.abspath(path))

def setup_java(p, c):
    gateway, jvm, app = connect_jvm(6006)
    app.buildIndex(get_java_file(jvm, "{}.txt".format(p)), get_java_file(jvm, "{}.fst".format(p)))
    app.buildIndex(get_java_file(jvm, "{}.txt".format(c)), get_java_file(jvm, "{}.fst".format(c)))

    indx = app.loadIndex(get_java_file(jvm, "{}.fst".format(p)))
    indx2 = app.loadIndex(get_java_file(jvm, "{}.fst".format(c)))
    
    return (app, indx, indx2)

# Create a document and tag it
def create_doc(id, text, app, i, i2):
    doc = Document()
    doc.add_text("main", text)  
    doc.props["id"] = id 
    binary_doc = MsgpackCodec.encode(doc) 
    search_binary_doc = app.search(i, i2, binary_doc)
    doc = MsgpackCodec.decode(search_binary_doc)
    return doc

def filter_away_nodes(d, stop_w):
    rm_nodes = []
    for node in d['protmatches']:
        #If node is a stopword or number or length less than 2
        if len(str(node["text"])) < 2 or re.match( "^[0-9-]+$", str(node["text"])) or str(node["text"]) in stop_w:
            rm_nodes.append(node)
    [n.detach() for n in rm_nodes]
    return d
    
#Taggs the abstracts in abstracts_genetag
def file_tagger(app, indx, indx2, stop_w):
    # Add the abstracts[key = PMID] = Abstract Text
    abstracts = {}
    with open('genetag/abstracts_genetag.txt', 'r',encoding="utf-8", errors='ignore') as file:
        abstracts = json.loads(file.read())

    # One doc for each abstract
    out = []; dr = []
    for key in abstracts:
        dr.append(create_doc(key, abstracts[key], app, indx, indx2))
    for d in dr:
        d = filter_away_nodes(d, stop_w)
    
    # Print one file per 50 000 abstracts
    i = 0; j = 0
    while i < len(dr):
        with DocumentIO.write('pubmed/pubmed1905({}).docria'.format(str(j))) as dw:
            while i < len(dr) and i < 50000:
                dw.write(dr[i])
                i += 1
        j += 1
    return len(dr)


# Tags a given string, s, and prints matches and returns new string, s_out
def text_tagger(s, app, indx, indx2):
   
    doc = filter_away_nodes(create_doc('id', s, app, indx, indx2))

    tuls = []
    for term in doc["protmatches"]:
        tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
    
    tuls2 = []
    for term in doc["lysomatches"]:
        tuls2.append((term["text"].start, term["text"].stop, str(term["text"])))
        
    i = 0; s_out = ''

    for d in tuls:
        print('------------------\nMATCH, PROT:\n{}\n------------------'.format(d[2]))
        i_s = d[0]
        i_e = d[1]
        s_out += s[i:i_s] + "\033[43m{}\033[m".format(s[i_s:i_e])
        i = i_e
        
    s_out += s[i:]

    for d in tuls2:
        print('------------------\nMATCH, LYSO:\n{}\n------------------'.format(d[2]))
        s_out = re.sub(d[2], "\033[45m{}\033[m".format(d[2]), s_out, flags = re.IGNORECASE) 
    return s_out

#A tagger which prints to genetag in eval format
def score_tagger(c):
    p = 'protein_name/protein_names_uniprot(2)'
    
    (app, indx, indx2) = setup_java(p, c)
    
    # Read dictionary with genetag abstracts 
    with open('genetag/abstracts_genetag.txt', 'r',encoding="utf-8", errors='ignore') as file:
        d = json.loads(file.read())

    # Write to genetag
    out = []

    for key in d:
        doc = filter_away_nodes(create_doc(key, d[key], app, indx, indx2))
        tuls = []
        
        for term in doc["protmatches"]:
            tuls.append((term["text"].start, term["text"].stop, str(term["text"])))

        for m in tuls:
            out.append('|'.join([key, str(m[0]) + ' ' + str(m[1]), m[2]]))

    with open('genetag/genetag_all.out', 'w+', encoding="utf-8", errors='ignore') as file:
        file.write('\n'.join(out))
        
    return len(d)

p = 'protein_name/protein_names_uniprot(2)'
c = 'cell_death_names'

stop_w = set()
[stop_w.add(t) for t in stopwords.words('english')]

(app, indx, indx2) = setup_java(p, c)

#file_tagger = 'file', text_tagger = 'text',  score_tagger = 'score'
run = 'file'

if run == 'file':
    start = time.time()
    l = file_tagger(app, indx, indx2, stop_w)
    end = time.time()
    print('total time: {} s\nnbr of abstracts: {}\ntime per abstract: {} s'.format(str(end-start), l, str((end-start)/l)))

elif run == 'text':
    #text = 'Improvement of nursing instruction to be given at the time of discharge from the ward for premature infants'
    text = "the apoptosis the. The constraints of Cr10HGO, 110 kDa antigen, and the effects of additional proteins on oligoribonucleotide synthesis by the 63-kDa gene 4 protein have been examined using templates of defined sequence. cell death is caused by cells dying."
    s = text_tagger(text, app, indx, indx2)
    matches = "{} \n\n\033[43m{}\033[m\n\033[45m{}\033[m".format(s, 'PROTEIN', 'CELL DEATH')
    print(matches)

elif run == 'score':
    start = time.time()
    l = score_tagger(c)
    end = time.time()
    print('total time: {} s\ntime per abstract: {} s'.format(str(end-start), str((end-start)/l)))

### Meeting with Marcus:
* Our stopword filter is making things crazy, should be done in post instead
* Use group by span in one of the layers in java
* Use remove to delete layers

* Without stopwordfilterfactory:
```
Accepting arbitrarily chosen limits of maximized errors of +/- 10%, it could be shown that the system did not work acceptably when the mean carbon dioxide concentration was below 1.5 vol.% within the fresh gas flow rates (2.2--7.7 1 min-1) and the range of minute ventilation (4--10 1 min-1) employed.{'id': 'P00155973A0387'}--------------------
```
* With stopfilterfactory

```
Accepting arbitrarily chosen limits of maximized errors of +/- 10%, it could be shown that the system did not work acceptably when the mean carbon dioxide concentration was below 1.5 vol.% within the fresh gas flow rates (2.2--7.7 1 min-1) and the range of minute ventilation (4--10 1 min-1) employed.{'id': 'P00155973A0387'}          Node protmatches#0          

  field    value                      

  id       42401                      
  terms    [Node<terms#53>]           
  text     span(main[231:232]) = '1'  
          Node protmatches#1           

  field    value                       

  id       600341                      
  terms    [Node<terms#53>]            
  text     span(main[236:238]) = '-1'  
          Node protmatches#2          

  field    value                      

  id       42401                      
  terms    [Node<terms#53>]           
  text     span(main[283:284]) = '1'  
          Node protmatches#3           

  field    value                       

  id       600341                      
  terms    [Node<terms#53>]            
  text     span(main[288:290]) = '-1'  
-------------------- 
```


## Day 14, 21/5
* Run through all pubmed abstracts and run the code on in
* Write function to connect protein name to id -> maybe fix write to docria
* Fix so that you can run through terminal and send files as input
* Start looking at report

In [ ]:
#From xml.gz files to .xml 
import os
import gzip
import shutil

path = 'pubmed2018/pubmed/'

directory = os.fsencode(path)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.xml.gz'):
        with gzip.open(path + filename, 'rb') as f_in:
            with open(path + filename[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

In [ ]:
#Read xml files to json dictionaries
from xml.etree import ElementTree as ET
import json
import os

path = 'pubmed2018/pubmed/'

directory = os.fsencode(path)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
    if filename.endswith('.xml'):
        parser = ET.iterparse(path + filename)
        id = ''
        abbe = {}
        for event, element in parser:
            tag = element.tag
            if tag == 'PMID':
                id = element.text
            elif tag == 'AbstractText':
                abbe.setdefault(id, []).append(element.text)
            element.clear()

        with open('pubmed/json_{}.txt'.format(filename[:-4]), 'w+',encoding="utf-8", errors='ignore') as file:
            file.write(json.dumps(abbe))

2min 10 sec for 10 files => 12 s per each pubmed.xml
We have 972 in total => 3,4 hours

In [ ]:
# Read dictionary with genetag abstracts 
with open('pubmed/json_pubmed19n0111.xml.txt', 'r',encoding="utf-8", errors='ignore') as file:
    d = json.loads(file.read())
i = 0
for key in d:
    if i > 10:
        break
    print(key)
    print(d[key])

In [ ]:
#Process protein names and cell death names using docria and print docria to file

import json
import re
import time
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.algorithm import group_by_span, dominant_right_span, dominant_right
from docria.printout import options
from docria.storage import DocumentIO
from nltk.corpus import stopwords
import os


set_large_screen()

def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app

def get_java_file(jvm, path):
        return jvm.java.io.File(os.path.abspath(path))

def setup_java(p, c):
    gateway, jvm, app = connect_jvm(6006)
    app.buildIndex(get_java_file(jvm, "{}.txt".format(p)), get_java_file(jvm, "{}.fst".format(p)))
    app.buildIndex(get_java_file(jvm, "{}.txt".format(c)), get_java_file(jvm, "{}.fst".format(c)))

    indx = app.loadIndex(get_java_file(jvm, "{}.fst".format(p)))
    indx2 = app.loadIndex(get_java_file(jvm, "{}.fst".format(c)))
    
    return (app, indx, indx2)

# Create a document and tag it
def create_doc(id, text, app, i, i2):
    doc = Document()
    doc.add_text("main", text)  
    doc.props["id"] = id 
    binary_doc = MsgpackCodec.encode(doc)
    search_binary_doc = app.search(i, i2, binary_doc)
    doc = MsgpackCodec.decode(search_binary_doc)
    return doc

def filter_away_nodes(d, stop_w):
    rm_nodes = []
    for node in d['protmatches']:
        #If node is a stopword or number or length less than 2
        if len(str(node["text"])) < 2 or re.match( "^[0-9-]+$", str(node["text"])) or str(node["text"]) in stop_w:
            rm_nodes.append(node)
    [n.detach() for n in rm_nodes]
    return d
    
#Taggs the abstracts in abstracts_genetag
def file_tagger(app, indx, indx2, stop_w, abstracts, filename):
    # One doc for each abstract
    out = []; dr = []
    for key in abstracts:
        text = ''.join(abstracts[key])
        dr.append(create_doc(key, text, app, indx, indx2))
    with DocumentIO.write('docria/out_{}.docria'.format(filename)) as dw:
        for d in dr:
            d = filter_away_nodes(d, stop_w)
            dw.write(d)
            
    return len(dr)

p = 'protein_name/protein_names_uniprot(2)'
c = 'cell_death_names'
path = 'pubmed/'


stop_w = set()
[stop_w.add(t) for t in stopwords.words('english')]

(app, indx, indx2) = setup_java(p, c)

directory = os.fsencode(path)


for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if '19n0001' in filename or '19n0002' in filename:
        continue
    if filename.endswith('.xml.txt'):
        with open(path + filename, 'r',encoding="utf-8", errors='ignore') as file:
            start = time.time()
            d = json.loads(file.read())
            l = file_tagger(app, indx, indx2, stop_w, d, filename)
            end = time.time()
            print('total time: {} s\nnbr of abstracts: {}\ntime per abstract: {} s'.format(str(end-start), l, str((end-start)/l)))

```total time: 171.7357325553894 s
nbr of abstracts: 15401
time per abstract: 0.011150946857696863 s ```

```total time: 144.01137924194336 s
nbr of abstracts: 13455
time per abstract: 0.010703186863020688 s ```

```total time: 129.37624216079712 s
nbr of abstracts: 12659
time per abstract: 0.01022009970462099 s```

```total time: 163.13451719284058 s
nbr of abstracts: 14984
time per abstract: 0.010887247543569178 s```

```total time: 158.24822568893433 s
nbr of abstracts: 14135
time per abstract: 0.011195488198721919 s```

```total time: 192.68476223945618 s
nbr of abstracts: 16385
time per abstract: 0.011759826807412645 s```

```total time: 191.50953817367554 s
nbr of abstracts: 16407
time per abstract: 0.011672428730034469 s```

```total time: 190.79297733306885 s
nbr of abstracts: 15397
time per abstract: 0.01239156831415658 s```

```total time: 66.98976349830627 s
nbr of abstracts: 5575
time per abstract: 0.012016101075929377 s```

```total time: 102.18244314193726 s
nbr of abstracts: 9382
time per abstract: 0.010891328409927228 s```

### Summary about work since last meeting:
We have
* Collected all names from uniprot, only euc

* Changed the filters in the java files and instead did it all in post processing as using the java files led to unexpected indexing. poS tagger/spacey -> solution takes too long/complicated, solved by checking if stopwords and lengthfilter which seemd to work great (see output)

* Started benchmarking time -> takes ~14s to read a pubmed.xml file + ~40s to process with docria = ~1 min

* Docria output has one layer for lyso matches and one for prot matches. Should we append genetag names to this?

* Wrote a method that takes a protein name and outputs the genetag id (perhaps add to docria)

* Started to process all the abstracts

* Time to run everything; almost 2 days